# Understanding Cicles in the Brazilian Stock Market

In [1]:
import MetaTrader5 as mt5
import pandas as pd

In [2]:
mt5.initialize()

login = 3014967797
password = 'kSS2FF5#'
server = 'Rico-DEMO'

mt5.login(login, password, server)

symbol = "BBAS3F"

timeframe = mt5.TIMEFRAME_D1
df = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, 0, 1000))
df['time'] = pd.to_datetime(df['time'], unit='s') 

In [3]:
df

,time,open,high,low,close,tick_volume,spread,real_volume
0,2020-09-22,30.65,31.67,30.65,30.83,5432,1,82244
1,2020-09-23,31.00,31.15,30.10,30.15,8413,1,121569
2,2020-09-24,30.18,30.90,29.91,30.45,4644,1,75583
3,2020-09-25,30.37,30.48,29.99,30.32,5015,1,73746
4,2020-09-28,30.93,31.75,30.11,30.50,5421,1,105601
...,...,...,...,...,...,...,...,...
995,2024-09-19,28.38,28.48,27.95,27.95,21138,1,228408
996,2024-09-20,27.90,27.91,27.47,27.52,30886,1,357325
997,2024-09-23,27.52,27.68,27.29,27.40,30673,1,284038
998,2024-09-24,27.72,27.91,27.38,27.51,21380,1,229260


In [4]:
data = {
        'Symbol': mt5.symbol_info(symbol).name,
        'Bid': mt5.symbol_info(symbol).bid,
        'Ask': mt5.symbol_info(symbol).ask,
        'Last Price': mt5.symbol_info(symbol).last,
        'Session Open': mt5.symbol_info(symbol).session_open,
        'Session Close': mt5.symbol_info(symbol).session_close,
        'Volume': mt5.symbol_info(symbol).volume,
        'Volume Real': mt5.symbol_info(symbol).volume_real,
        'Price Change': mt5.symbol_info(symbol).price_change,
        'Price Volatility': mt5.symbol_info(symbol).price_volatility,
    }

    # Criar o DataFrame
df_current_data = pd.DataFrame(data, index=[0])

In [5]:
df_current_data 

,Symbol,Bid,Ask,Last Price,Session Open,Session Close,Volume,Volume Real,Price Change,Price Volatility
0,BBAS3F,27.44,27.45,27.45,27.3,27.38,24,24.0,0.2557,0.0
